# StructuredTool

The StructuredTool.from_function class method provides a bit more configurability than the @tool decorator, without requiring much additional code.


#### guide:

https://python.langchain.com/v0.2/docs/how_to/custom_tools/#structuredtool

In [1]:
from langchain.pydantic_v1 import BaseModel, Field

class CalculatorInput(BaseModel):
    a: int = Field(description="first number")
    b: int = Field(description="second number")

In [2]:
from langchain_core.tools import StructuredTool

# Define the functions for each operation
def add(a: int, b: int) -> int:
    """Add two numbers."""
    return a + b

def subtract(a: int, b: int) -> int:
    """Subtract two numbers."""
    return a - b

def divide(a: int, b: int) -> float:
    """Divide two numbers."""
    if b == 0:
        raise ValueError("Cannot divide by zero")
    return a / b

def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

# Define the asynchronous versions of the functions
async def aadd(a: int, b: int) -> int:
    """Add two numbers."""
    return a + b

async def asubtract(a: int, b: int) -> int:
    """Subtract two numbers."""
    return a - b

async def adivide(a: int, b: int) -> float:
    """Divide two numbers."""
    if b == 0:
        raise ValueError("Cannot divide by zero")
    return a / b

async def amultiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b


In [3]:
add_tool = StructuredTool.from_function(
    func=add,
    name="add",
    description="Add 2 numbers",
    args_schema=CalculatorInput,
    return_direct=False,
    coroutine=aadd
)

subtract_tool = StructuredTool.from_function(
    func=subtract,
    name="subtract",
    description="Subtract 2 numbers",
    args_schema=CalculatorInput,
    return_direct=False,
    coroutine=asubtract
)


multiply_tool = StructuredTool.from_function(
    func=multiply,
    name="multiply",
    description="Multiply 2 numbers",
    args_schema=CalculatorInput,
    return_direct=False,
    coroutine=amultiply
)


divide_tool = StructuredTool.from_function(
    func=divide,
    name="divide",
    description="Divide 2 numbers",
    args_schema=CalculatorInput,
    return_direct=False,
    coroutine=adivide
)




In [4]:

# Example usage
print(add_tool.invoke({"a": 2, "b": 3}))  # Output: 5
print(await subtract_tool.ainvoke({"a": 10, "b": 4}))  # Output: 6
print(divide_tool.invoke({"a": 8, "b": 2}))  # Output: 4.0
print(await multiply_tool.ainvoke({"a": 2, "b": 5}))  # Output: 10


5
6
4.0
10


In [5]:
tools = [add_tool, subtract_tool, multiply_tool, divide_tool]

In [6]:
from langchain import hub

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

c:\Code\Github\LangChain-Basics\venv\lib\site-packages\langchain\hub.py:86: DeprecationWarning: The `langchainhub sdk` is deprecated.
Please use the `langsmith sdk` instead:
  pip install langsmith
Use the `pull_prompt` method.
  res_dict = client.pull_repo(owner_repo_commit)


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [7]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = llm.bind_tools(tools)

In [8]:
from langchain.agents import create_tool_calling_agent

agent = create_tool_calling_agent(llm_with_tools, tools, prompt)

In [9]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [10]:
agent_executor.invoke({"input": "What is 3 +78 and what is 67*89"})



> Entering new AgentExecutor chain...

Invoking: `add` with `{'a': 3, 'b': 78}`


81
Invoking: `multiply` with `{'a': 67, 'b': 89}`


5963The result of \(3 + 78\) is \(81\), and the result of \(67 \times 89\) is \(5963\).

> Finished chain.


{'input': 'What is 3 +78 and what is 67*89',
 'output': 'The result of \\(3 + 78\\) is \\(81\\), and the result of \\(67 \\times 89\\) is \\(5963\\).'}

In [11]:
agent_executor.invoke({"input": "if i have 3 apples and i get 567 more apples. How many apples will I have in all?"})



> Entering new AgentExecutor chain...

Invoking: `add` with `{'a': 3, 'b': 567}`


570If you have 3 apples and you get 567 more apples, you will have a total of 570 apples.

> Finished chain.


{'input': 'if i have 3 apples and i get 567 more apples. How many apples will I have in all?',
 'output': 'If you have 3 apples and you get 567 more apples, you will have a total of 570 apples.'}